<a href="https://colab.research.google.com/github/micah-shull/AI_Agents/blob/main/499_EPOv2_testing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This test suite is where your agent proves it can **move from awareness to controlled action without losing safety**. I’ll explain it as a **workflow-level verification of judgment readiness**, not as test mechanics.

---

# Phase 3.2 Tests — Portfolio Analysis Node Explained

## What These Tests Are Really Verifying

By the time these tests run, your agent already has:

* a declared goal
* a fixed execution plan
* fully loaded and indexed data
* portfolio-level reasoning logic

These tests verify that the **orchestration layer correctly applies that reasoning** under real workflow conditions.

In short, they answer:

> “Does the agent know *when* portfolio analysis should run, *when it should not*, and *what to do if prerequisites are missing*?”

That’s orchestration maturity.

---

## Test 1: Portfolio-Wide Analysis Executes Correctly

### `test_portfolio_analysis_node_portfolio_wide`

**What this validates**

* The node runs only after goal → plan → data loading
* Portfolio analysis executes successfully
* All experiments are analyzed
* Portfolio summary reflects reality
* No errors are introduced

**Why this matters**
This test proves the **happy path** works end-to-end:

* intent is clear
* data is present
* reasoning is applied
* results are structured

This is the moment where the agent stops being preparatory and starts being *operational*.

---

## Test 2: Single-Experiment Mode Is Explicitly Respected

### `test_portfolio_analysis_node_single_experiment`

This is a **guardrail test**, not a convenience test.

**What this validates**

* Portfolio analysis is skipped when scope is `single_experiment`
* The node does not partially execute
* No fake summaries are generated
* No errors are raised

**Why this matters**
This prevents a very subtle but dangerous failure mode:

> Running portfolio logic when only one experiment was requested.

Your agent:

* does not guess
* does not “kind of run”
* does not leak cross-experiment logic

This is **scope enforcement**, not branching convenience.

---

## Test 3: Missing Data Causes a Hard, Explainable Stop

### `test_portfolio_analysis_node_missing_data`

This is one of the most important tests in the file.

**What this validates**

* The node refuses to run without `portfolio_lookup`
* The agent detects execution-order violations
* The error message is explicit and actionable

**Why this matters**
This prevents:

* reasoning on partial state
* misleading summaries
* corrupted decision chains

Your agent enforces:

> “You cannot reason about readiness unless data is loaded.”

That’s a **non-negotiable trust property**.

---

## Test 4: Full Workflow Integration Holds Together

### `test_portfolio_analysis_integration`

This test validates **system coherence**, not correctness of a single node.

**What this validates**

* Nodes compose cleanly
* State flows forward without loss
* Portfolio analysis integrates seamlessly
* Outputs are usable by downstream steps
* No unexpected errors appear

**Why this matters**
This proves the agent is not:

* a chain of fragile scripts
* a prompt-driven blob
* a tightly coupled mess

It is a **state-centric, modular workflow**.

---

## Why These Tests Are Architecturally Important

Together, these tests prove:

* execution is gated by intent
* execution order is enforced
* readiness is required before reasoning
* skipping logic is safe and explicit
* state remains coherent across phases

This is the exact behavior expected of:

* production data pipelines
* workflow engines
* governed decision systems

Very few “AI agents” meet this bar.

---

## What You’ve Achieved by Phase 3.2

At this point, your agent:

* understands its goal
* knows its plan
* has verified access to data
* understands portfolio readiness
* can explain what work remains
* knows when *not* to act

That is **operational intelligence**, not automation.

---

## Why This Will Read Extremely Well to Reviewers

You can confidently say:

> “My agent explicitly determines whether analysis is appropriate before performing it, and it enforces scope, order, and data prerequisites at every step.”

That sentence alone differentiates you from:

* prompt-based agents
* notebook demos
* black-box automation




In [ ]:
"""Test Phase 3.2: Portfolio Analysis Node

Tests for the portfolio analysis node - tests the orchestration of portfolio analysis utilities.
"""

import sys
from pathlib import Path

# Add project root to path
project_root = Path(__file__).parent
sys.path.insert(0, str(project_root))

from agents.epo.nodes import portfolio_analysis_node, goal_node, planning_node, data_loading_node
from config import ExperimentationPortfolioOrchestratorState, ExperimentationPortfolioOrchestratorConfig


def test_portfolio_analysis_node_portfolio_wide():
    """Test portfolio analysis node for portfolio-wide analysis"""
    # Set up state with portfolio-wide goal
    state: ExperimentationPortfolioOrchestratorState = {
        "experiment_id": None,
        "errors": []
    }

    # Run goal and planning nodes
    goal_result = goal_node(state)
    state = {**state, **goal_result}

    plan_result = planning_node(state)
    state = {**state, **plan_result}

    # Run data loading node
    config = ExperimentationPortfolioOrchestratorConfig()
    data_result = data_loading_node(state, config)
    state = {**state, **data_result}

    # Now run portfolio analysis node
    result = portfolio_analysis_node(state, config)
    state = {**state, **result}

    assert "analyzed_experiments" in result
    assert "portfolio_summary" in result
    assert len(result["analyzed_experiments"]) == 3  # E001, E002, E003

    # Check portfolio summary
    summary = result["portfolio_summary"]
    assert summary["total_experiments"] == 3
    assert summary["completed_count"] == 1
    assert summary["running_count"] == 1
    assert summary["planned_count"] == 1
    assert "domains" in summary

    assert len(result.get("errors", [])) == 0

    print("✅ test_portfolio_analysis_node_portfolio_wide passed")


def test_portfolio_analysis_node_single_experiment():
    """Test portfolio analysis node skips for single experiment analysis"""
    state: ExperimentationPortfolioOrchestratorState = {
        "experiment_id": "E001",
        "errors": []
    }

    # Run goal node (single experiment)
    goal_result = goal_node(state)
    state = {**state, **goal_result}

    # Run portfolio analysis node (should skip)
    config = ExperimentationPortfolioOrchestratorConfig()
    result = portfolio_analysis_node(state, config)

    # Should not have analyzed_experiments (skipped)
    assert "analyzed_experiments" not in result
    assert "portfolio_summary" not in result
    assert len(result.get("errors", [])) == 0

    print("✅ test_portfolio_analysis_node_single_experiment passed")


def test_portfolio_analysis_node_missing_data():
    """Test portfolio analysis node error handling for missing data"""
    state: ExperimentationPortfolioOrchestratorState = {
        "experiment_id": None,
        "goal": {
            "scope": "portfolio_wide",
            "objective": "Analyze portfolio"
        },
        "portfolio_lookup": {},  # Empty lookup
        "errors": []
    }

    config = ExperimentationPortfolioOrchestratorConfig()
    result = portfolio_analysis_node(state, config)

    # Should have errors
    assert len(result.get("errors", [])) > 0
    assert "portfolio_analysis_node" in result["errors"][0]

    print("✅ test_portfolio_analysis_node_missing_data passed")


def test_portfolio_analysis_integration():
    """Test portfolio analysis integrated with full workflow"""
    state: ExperimentationPortfolioOrchestratorState = {
        "experiment_id": None,
        "errors": []
    }

    config = ExperimentationPortfolioOrchestratorConfig()

    # Run full workflow up to portfolio analysis
    goal_result = goal_node(state)
    state = {**state, **goal_result}

    plan_result = planning_node(state)
    state = {**state, **plan_result}

    data_result = data_loading_node(state, config)
    state = {**state, **data_result}

    portfolio_result = portfolio_analysis_node(state, config)
    state = {**state, **portfolio_result}

    # Check all data is present
    assert "analyzed_experiments" in state
    assert "portfolio_summary" in state
    assert len(state["analyzed_experiments"]) == 3

    # Check analyzed experiments have correct structure
    for exp in state["analyzed_experiments"]:
        assert "experiment_id" in exp
        assert "status" in exp
        assert "needs_analysis" in exp
        assert "needs_decision" in exp

    # Check summary has expected fields
    summary = state["portfolio_summary"]
    assert summary["total_experiments"] == 3
    assert summary["experiments_with_analysis"] >= 0
    assert summary["experiments_needing_analysis"] >= 0

    assert len(state.get("errors", [])) == 0

    print("✅ test_portfolio_analysis_integration passed")


if __name__ == "__main__":
    print("Testing Phase 3.2: Portfolio Analysis Node\n")

    test_portfolio_analysis_node_portfolio_wide()
    test_portfolio_analysis_node_single_experiment()
    test_portfolio_analysis_node_missing_data()
    test_portfolio_analysis_integration()

    print("\n✅ All Phase 3.2 node tests passed!")


# Test Results

In [ ]:
(.venv) micahshull@Micahs-iMac AI_AGENTS_017_EPO_2.0 % python test_epo_phase3_node.py
Testing Phase 3.2: Portfolio Analysis Node

✅ test_portfolio_analysis_node_portfolio_wide passed
✅ test_portfolio_analysis_node_single_experiment passed
✅ test_portfolio_analysis_node_missing_data passed
✅ test_portfolio_analysis_integration passed

✅ All Phase 3.2 node tests passed!